This script performs various exploratory regression analyses to test the hypotheses as to whether the difference in enrollment patterns among different ethnic groups contributes to algorithm bias. In particular, this script also generates Appendix Table A8.

In [ ]:
test = pd.read_csv("D:\\Yifeng -- Project Work\\ys8mz_sandbox\\bias_analyses_8\\college_program_race.csv", low_memory=False)

In [9]:
test.columns.values

array(['vccsid', 'college', 'curr', 'strm', 'cip', 'deglvl', 'course',
       'section', 'grade', 'race'], dtype=object)

In [10]:
test.shape

(5107603, 10)

In [11]:
Counter(test.grade)

Counter({0: 1249188, 1: 3858415})

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import statsmodels.api as sm
from statsmodels.tools import add_constant

In [72]:
df1 = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\intermediate_files\\new\\full_data_truncated.dta")
df2 = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\intermediate_files\\new\\bias\\student_college_program.dta")

In [73]:
df3 = df1[np.array(df1.white == 1) | np.array(df1.afam == 1) | np.array(df1.hisp == 1) | np.array(df1.asian == 1) | np.array(df1.other == 1)]
df3.loc[:,'race'] = "white"
for r in ['afam', 'hisp', 'asian', 'other']:
    df3.loc[:,'race'] = df3.apply(lambda x: r if x[r] == 1 else x.race, axis=1)
df3 = df3.loc[:,['vccsid', 'race']]

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [74]:
df = df2.merge(df1.loc[:,['vccsid', 'grad_6years']], on=['vccsid'], how='inner').merge(df3, on=['vccsid'], how='inner')
df = df.rename(columns={'grad_6years': 'graduate'})
df.loc[:,'cip'] = df.cip.astype(int)
df.loc[:,'college'] = df.college.apply(lambda x: x.replace(" ", "_"))
df.sort_values(['vccsid']).to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\intermediate_files\\new\\bias\\college_program_race.csv", index=False)

In [75]:
results = pd.read_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\full_model_outputs.csv")
results.iloc[:,1] = results.iloc[:,1].round(4)
results.iloc[:,2] = results.iloc[:,2].round(4)
results.columns = ['predictor', 'coefficient', 'p_value']
results.loc[:,'predictor'] = results.predictor.apply(lambda x: x.replace(":", " * ").replace("_", " "))
results.loc[:,'predictor'] = results.predictor.apply(lambda x: x.replace('degree lvlD', 'diploma').replace('degree lvlC', 'certificate').replace('degree lvlA', 'associate'))
results.loc[:,'predictor'] = results.predictor.apply(lambda x: re.sub("^college", "", x))
results.loc[:,'predictor'] = results.predictor.apply(lambda x: x.replace("race", "").replace("(", "").replace(")", ""))
results.loc[:,'star'] = results.p_value.apply(lambda x: "**" if x < 0.05 else "")
results.loc[:,'p_value'] = results.p_value.astype(str) + results.star
results.loc[:,'p_value'] = results.p_value.apply(lambda x: x.replace("0.0**", "0**"))
results = results.drop(['star'], axis=1)
results.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\intermediate_files\\new\\bias\\full_model_outputs_clean.csv", index=False)

In [76]:
by_college_1 = df.groupby(['college', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='college', columns='race', values='graduate').reset_index().round(4)
by_college_1.loc[:,'college'] = by_college_1.college.apply(lambda x: x.replace("_", " "))
by_college_1.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\success_rate_by_college.csv", index=False)

In [77]:
by_college_2 = df.groupby(['college', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college', columns='race', values='vccsid').reset_index().round(4)
by_college_2.loc[:,'total'] = by_college_2.iloc[:,1:].sum(axis=1)
for v in by_college_2.columns.values[1:-1]:
    by_college_2.loc[:,v] = by_college_2[v] / by_college_2.total
by_college_2 = by_college_2.rename(columns = {'total': 'N'}).round(4)
by_college_2.loc[:,'college'] = by_college_1.college.apply(lambda x: x.replace("_", " "))
by_college_2 = by_college_2.loc[:,['college','N','afam','asian','hisp','other','white']]
by_college_2.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\enrollment_share_by_college.csv", index=False)

In [78]:
X = by_college_1.white
X = add_constant(X, prepend=True)
y = by_college_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                    0.1799
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.676
Time:                        11:56:51   Log-Likelihood:                 9.7771
No. Observations:                  23   AIC:                            -15.55
Df Residuals:                      21   BIC:                            -13.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0304      0.476      0.064      0.950      -0.959       1.020
white          0.5247      1.237      0.424      0.676      -2.048       3.097
==============================================================================
Omnibus:                        8.816   Durbin-Watson:                   1.746
Prob(Omnibus):                  0.012   Jarque-Bera (JB):                2.168
Skew:                           0.200   Prob(JB):                        0.338
Kurtosis:                       1.550   Cond. No.                         41.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
top15_cip = Counter(df.cip).most_common()
top15_cip = [t for t in top15_cip if t[0] != 0][:15]
top15_cip = pd.DataFrame(top15_cip, columns=['cip', 'N'])
by_cip_1 = df.merge(top15_cip, how='inner', on=['cip'])
by_cip_1 = by_cip_1.groupby(['cip', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='cip', columns='race', values='graduate').reset_index().round(4)
by_cip_1 = by_cip_1.merge(top15_cip, how='inner', on=['cip']).sort_values(['N'], ascending=False).loc[:,['cip','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_cip_1.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\success_rate_by_cip.csv", index=False)

In [80]:
by_cip_2 = df.merge(top15_cip, how='inner', on=['cip'])
by_cip_2 = by_cip_2.groupby(['cip', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='cip', columns='race', values='vccsid').reset_index().round(4)
by_cip_2.loc[:,'total'] = by_cip_2.iloc[:,1:].sum(axis=1)
for v in by_cip_2.columns.values[1:-1]:
    by_cip_2.loc[:,v] = by_cip_2[v] / by_cip_2.total
by_cip_2 = by_cip_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_cip_2 = by_cip_2.loc[:,['cip','N','afam','asian','hisp','other','white']]
by_cip_2.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\enrollment_share_by_cip.csv", index=False)

In [81]:
top_cip = [(t[0], t[1]/df.shape[0]) for t in Counter(df.cip).most_common()]
top_cip

[(24, 0.30958658720463417),
 (51, 0.1546647728627796),
 (52, 0.14824403571021622),
 (45, 0.06598546258191541),
 (30, 0.056827366172811374),
 (11, 0.04504850664200482),
 (43, 0.02969491561354741),
 (0, 0.029676366228459528),
 (15, 0.027924774293732163),
 (50, 0.02520331451012399),
 (47, 0.02428379499219601),
 (14, 0.02379356124344478),
 (19, 0.018790527094026833),
 (48, 0.008691711869751519),
 (12, 0.008167029262979932),
 (22, 0.0067122274896587175),
 (46, 0.005074581777614072),
 (49, 0.0028831044250883084),
 (1, 0.0021702780552824673),
 (16, 0.0020801810419984576),
 (4, 0.0018284393872343133),
 (13, 0.001489250631341571),
 (31, 0.0011792109091583614)]

In [82]:
top_cip = [t for t in top_cip if t[0] != 0]
top_cip = pd.DataFrame(top_cip, columns=['cip', 'N'])
by_cip_1 = df.merge(top_cip, how='inner', on=['cip'])
by_cip_1 = by_cip_1.groupby(['cip', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='cip', columns='race', values='graduate').reset_index().round(4)
by_cip_1 = by_cip_1.merge(top_cip, how='inner', on=['cip']).sort_values(['N'], ascending=False).loc[:,['cip','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_cip_2 = df.merge(top_cip, how='inner', on=['cip'])
by_cip_2 = by_cip_2.groupby(['cip', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='cip', columns='race', values='vccsid').reset_index().round(4)
by_cip_2.loc[:,'total'] = by_cip_2.iloc[:,1:].sum(axis=1)
for v in by_cip_2.columns.values[1:-1]:
    by_cip_2.loc[:,v] = by_cip_2[v] / by_cip_2.total
by_cip_2 = by_cip_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_cip_2 = by_cip_2.loc[:,['cip','N','afam','asian','hisp','other','white']]

In [83]:
X = by_cip_1.white
X = add_constant(X, prepend=True)
y = by_cip_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.045
Method:                 Least Squares   F-statistic:                   0.08879
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.769
Time:                        11:56:53   Log-Likelihood:                 24.330
No. Observations:                  22   AIC:                            -44.66
Df Residuals:                      20   BIC:                            -42.48
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2341      0.120      1.954      0.065      -0.016       0.484
white          0.1043      0.350      0.298      0.769      -0.626       0.834
==============================================================================
Omnibus:                        1.567   Durbin-Watson:                   2.248
Prob(Omnibus):                  0.457   Jarque-Bera (JB):                0.523
Skew:                           0.328   Prob(JB):                        0.770
Kurtosis:                       3.373   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [85]:
top_curr = Counter(df.curr).most_common()
top_curr = [t for t in top_curr if t[0] != "000"][:15]
top_curr = pd.DataFrame(top_curr, columns=['curr', 'N'])
by_curr_1 = df.merge(top_curr, how='inner', on=['curr'])
by_curr_1 = by_curr_1.groupby(['curr', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='curr', columns='race', values='graduate').reset_index().round(4)
by_curr_1 = by_curr_1.merge(top_curr, how='inner', on=['curr']).sort_values(['N'], ascending=False).loc[:,['curr','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_curr_1.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\success_rate_by_curr.csv", index=False)

In [86]:
by_curr_2 = df.merge(top15_curr, how='inner', on=['curr'])
by_curr_2 = by_curr_2.groupby(['curr', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='curr', columns='race', values='vccsid').reset_index().round(4)
by_curr_2.loc[:,'total'] = by_curr_2.iloc[:,1:].sum(axis=1)
for v in by_curr_2.columns.values[1:-1]:
    by_curr_2.loc[:,v] = by_curr_2[v] / by_curr_2.total
by_curr_2 = by_curr_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_curr_2 = by_curr_2.loc[:,['curr','N','afam','asian','hisp','other','white']]
by_curr_2.to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\enrollment_share_by_curr.csv", index=False)

In [87]:
top_curr = [(t[0], t[1]/df.shape[0]) for t in Counter(df.curr).most_common() if t[1] >= 100 and t[0] != "000"]
top_curr

[('699', 0.11178124445174642),
 ('213', 0.07650296392674583),
 ('697', 0.0662902024797878),
 ('882', 0.06598546258191541),
 ('880', 0.056827366172811374),
 ('156', 0.05534871519009145),
 ('648', 0.049357263806704806),
 ('212', 0.0361368520633541),
 ('649', 0.027670382726812607),
 ('190', 0.02573594685336181),
 ('831', 0.023427873365997918),
 ('299', 0.02207111834242695),
 ('400', 0.019972387915340607),
 ('636', 0.015923322141870998),
 ('881', 0.014934904907902303),
 ('203', 0.014415522125441541),
 ('216', 0.01425917730827223),
 ('480', 0.013138264466532934),
 ('340', 0.011696712253988781),
 ('624', 0.01133367428869733),
 ('157', 0.011251527011879555),
 ('650', 0.009727827522517628),
 ('909', 0.009261442983165108),
 ('298', 0.009105098165995798),
 ('146', 0.0071574127317679415),
 ('246', 0.0070381666847744),
 ('995', 0.006463135747049985),
 ('242', 0.005906654194413455),
 ('260', 0.00583510656621733),
 ('529', 0.005734409904311672),
 ('718', 0.005130229932877725),
 ('172', 0.00509048125

In [88]:
top_curr = pd.DataFrame(top_curr, columns=['curr', 'N'])
by_curr_1 = df.merge(top_curr, how='inner', on=['curr'])
by_curr_1 = by_curr_1.groupby(['curr', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='curr', columns='race', values='graduate').reset_index().round(4)
by_curr_1 = by_curr_1.merge(top_curr, how='inner', on=['curr']).sort_values(['N'], ascending=False).loc[:,['curr','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_curr_2 = df.merge(top_curr, how='inner', on=['curr'])
by_curr_2 = by_curr_2.groupby(['curr', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='curr', columns='race', values='vccsid').reset_index().round(4)
by_curr_2.loc[:,'total'] = by_curr_2.iloc[:,1:].sum(axis=1)
for v in by_curr_2.columns.values[1:-1]:
    by_curr_2.loc[:,v] = by_curr_2[v] / by_curr_2.total
by_curr_2 = by_curr_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_curr_2 = by_curr_2.loc[:,['curr','N','afam','asian','hisp','other','white']]

In [89]:
X = by_curr_1.white
X = add_constant(X, prepend=True)
y = by_curr_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.346
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.249
Time:                        11:59:31   Log-Likelihood:                 64.611
No. Observations:                  82   AIC:                            -125.2
Df Residuals:                      80   BIC:                            -120.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3174      0.048      6.571      0.000       0.221       0.413
white         -0.1573      0.136     -1.160      0.249      -0.427       0.112
==============================================================================
Omnibus:                        3.070   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.215   Jarque-Bera (JB):                3.008
Skew:                           0.455   Prob(JB):                        0.222
Kurtosis:                       2.770   Cond. No.                         12.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [90]:
df = df.rename(columns = {'degree_lvl': 'deglvl'})

In [ ]:
by_deglvl_1 = df.groupby(['deglvl', 'race']).agg({'grade':'mean'}).reset_index().pivot(index='deglvl', columns='race', values='grade').reset_index().round(4)
by_deglvl_1.loc[:,'deglvl'] = by_deglvl_1.deglvl.apply(lambda x: x.replace("_", " "))
by_deglvl_1[by_deglvl_1.deglvl != "N"].to_csv("success_rate_by_deglvl.csv", index=False)
by_deglvl_2 = df.groupby(['deglvl', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='deglvl', columns='race', values='vccsid').reset_index().round(4)
by_deglvl_2.loc[:,'total'] = by_deglvl_2.iloc[:,1:].sum(axis=1)
for v in by_deglvl_2.columns.values[1:-1]:
    by_deglvl_2.loc[:,v] = by_deglvl_2[v] / by_deglvl_2.total
by_deglvl_2 = by_deglvl_2.rename(columns = {'total': 'N'}).round(4)
by_deglvl_2.loc[:,'deglvl'] = by_deglvl_1.deglvl.apply(lambda x: x.replace("_", " "))
by_deglvl_2 = by_deglvl_2.loc[:,['deglvl','N','afam','asian','hisp','other','white']]
by_deglvl_2[by_deglvl_2.deglvl != "N"].to_csv("enrollment_share_by_deglvl.csv", index=False)

In [91]:
by_deglvl_1 = df.groupby(['deglvl', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='deglvl', columns='race', values='graduate').reset_index().round(4)
by_deglvl_1.loc[:,'deglvl'] = by_deglvl_1.deglvl.apply(lambda x: x.replace("_", " "))
by_deglvl_1[by_deglvl_1.deglvl != "N"].to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\success_rate_by_deglvl.csv", index=False)
by_deglvl_2 = df.groupby(['deglvl', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='deglvl', columns='race', values='vccsid').reset_index().round(4)
by_deglvl_2.loc[:,'total'] = by_deglvl_2.iloc[:,1:].sum(axis=1)
for v in by_deglvl_2.columns.values[1:-1]:
    by_deglvl_2.loc[:,v] = by_deglvl_2[v] / by_deglvl_2.total
by_deglvl_2 = by_deglvl_2.rename(columns = {'total': 'N'}).round(4)
by_deglvl_2.loc[:,'deglvl'] = by_deglvl_1.deglvl.apply(lambda x: x.replace("_", " "))
by_deglvl_2 = by_deglvl_2.loc[:,['deglvl','N','afam','asian','hisp','other','white']]
by_deglvl_2[by_deglvl_2.deglvl != "N"].to_csv("C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\bias\\new\\enrollment_share_by_deglvl.csv", index=False)

In [92]:
X = by_deglvl_1.white
X = add_constant(X, prepend=True)
y = by_deglvl_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     10.79
Date:                Mon, 16 May 2022   Prob (F-statistic):             0.0815
Time:                        12:00:11   Log-Likelihood:                 10.393
No. Observations:                   4   AIC:                            -16.79
Df Residuals:                       2   BIC:                            -18.01
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4693      0.072      6.481      0.023       0.158       0.781
white         -0.5889      0.179     -3.285      0.082      -1.360       0.182
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.043
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.393
Skew:                          -0.097   Prob(JB):                        0.822
Kurtosis:                       1.476   Cond. No.                         16.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
df.loc[:,'college_cip'] = df.college + " - " + df.cip.astype(str)
df.loc[:,'college_curr'] = df.college + " - " + df.curr

In [94]:
top_college_cip = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_cip).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
top_college_cip

[('Northern_Virginia - 24', 0.09499140103505568),
 ('Northern_Virginia - 52', 0.056572974605891814),
 ('Tidewater - 51', 0.034316362412586024),
 ('Tidewater - 52', 0.030447490665684432),
 ('Tidewater - 45', 0.02587639219759865),
 ('Tidewater - 24', 0.025677648785942746),
 ('Germanna - 24', 0.02371671379093783),
 ('Northern_Virginia - 11', 0.023279478285294843),
 ('John_Tyler - 24', 0.020332775968476643),
 ('J._Sargeant_Reynolds - 51', 0.019988287388273077),
 ('Lord_Fairfax - 24', 0.01926221145769018),
 ('Tidewater - 30', 0.018766677884628123),
 ('Northern_Virginia - 45', 0.018671281047033292),
 ('Northern_Virginia - 30', 0.01705218472007653),
 ('Central_Virginia - 24', 0.015597382946755315),
 ('Blue_Ridge - 24', 0.014020685214285147),
 ('Thomas_Nelson - 52', 0.013904089079447016),
 ('J._Sargeant_Reynolds - 52', 0.013278709810769772),
 ('New_River - 24', 0.01182655794960397),
 ('Northern_Virginia - 50', 0.011405221916893455),
 ('Thomas_Nelson - 51', 0.010631447567513138),
 ('Thomas_Nels

In [95]:
top_college_cip = pd.DataFrame(top_college_cip, columns=['college_cip', 'N'])
by_college_cip_1 = df.merge(top_college_cip, how='inner', on=['college_cip'])
by_college_cip_1 = by_college_cip_1.groupby(['college_cip', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='college_cip', columns='race', values='graduate').reset_index().round(4)
by_college_cip_1 = by_college_cip_1.merge(top_college_cip, how='inner', on=['college_cip']).sort_values(['N'], ascending=False).loc[:,['college_cip','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_cip_2 = df.merge(top_college_cip, how='inner', on=['college_cip'])
by_college_cip_2 = by_college_cip_2.groupby(['college_cip', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_cip', columns='race', values='vccsid').reset_index().round(4)
by_college_cip_2.loc[:,'total'] = by_college_cip_2.iloc[:,1:].sum(axis=1)
for v in by_college_cip_2.columns.values[1:-1]:
    by_college_cip_2.loc[:,v] = by_college_cip_2[v] / by_college_cip_2.total
by_college_cip_2 = by_college_cip_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_cip_2 = by_college_cip_2.loc[:,['college_cip','N','afam','asian','hisp','other','white']].fillna(0)

In [96]:
X = by_college_cip_1.white
X = add_constant(X, prepend=True)
y = by_college_cip_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.355
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.246
Time:                        12:00:22   Log-Likelihood:                 79.422
No. Observations:                 227   AIC:                            -154.8
Df Residuals:                     225   BIC:                            -148.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1991      0.042      4.747      0.000       0.116       0.282
white          0.1357      0.117      1.164      0.246      -0.094       0.365
==============================================================================
Omnibus:                       14.173   Durbin-Watson:                   2.159
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               13.800
Skew:                           0.551   Prob(JB):                      0.00101
Kurtosis:                       2.508   Cond. No.                         11.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [97]:
top_college_curr = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_curr).most_common() if t[1] >= 100 and t[0].endswith(" - 000") == False]
top_college_curr

[('Northern_Virginia - 699', 0.067699955746467),
 ('Northern_Virginia - 213', 0.029713464998635294),
 ('Northern_Virginia - 648', 0.02727819572781162),
 ('Tidewater - 882', 0.02587639219759865),
 ('Tidewater - 213', 0.02046792148840266),
 ('Tidewater - 880', 0.018766677884628123),
 ('Northern_Virginia - 882', 0.018671281047033292),
 ('Northern_Virginia - 880', 0.01705218472007653),
 ('Tidewater - 699', 0.017012436037745347),
 ('Tidewater - 156', 0.015395989622943999),
 ('Northern_Virginia - 212', 0.015297942873193754),
 ('Blue_Ridge - 649', 0.013983586444109377),
 ('Germanna - 697', 0.013941187849622784),
 ('John_Tyler - 699', 0.012125998023165533),
 ('Central_Virginia - 697', 0.010949437026162582),
 ('Northern_Virginia - 340', 0.010822241242702804),
 ('Thomas_Nelson - 882', 0.010514851432675007),
 ('J._Sargeant_Reynolds - 213', 0.009603281651213262),
 ('Thomas_Nelson - 213', 0.0090838988687525),
 ('J._Sargeant_Reynolds - 156', 0.009038850362110495),
 ('Thomas_Nelson - 880', 0.00838962

In [98]:
top_college_curr = pd.DataFrame(top_college_curr, columns=['college_curr', 'N'])
by_college_curr_1 = df.merge(top_college_curr, how='inner', on=['college_curr'])
by_college_curr_1 = by_college_curr_1.groupby(['college_curr', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='college_curr', columns='race', values='graduate').reset_index().round(4)
by_college_curr_1 = by_college_curr_1.merge(top_college_curr, how='inner', on=['college_curr']).sort_values(['N'], ascending=False).loc[:,['college_curr','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_curr_2 = df.merge(top_college_curr, how='inner', on=['college_curr'])
by_college_curr_2 = by_college_curr_2.groupby(['college_curr', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_curr', columns='race', values='vccsid').reset_index().round(4)
by_college_curr_2.loc[:,'total'] = by_college_curr_2.iloc[:,1:].sum(axis=1)
for v in by_college_curr_2.columns.values[1:-1]:
    by_college_curr_2.loc[:,v] = by_college_curr_2[v] / by_college_curr_2.total
by_college_curr_2 = by_college_curr_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_curr_2 = by_college_curr_2.loc[:,['college_curr','N','afam','asian','hisp','other','white']].fillna(0)

In [99]:
X = by_college_curr_1.white
X = add_constant(X, prepend=True)
y = by_college_curr_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     15.17
Date:                Mon, 16 May 2022   Prob (F-statistic):           0.000114
Time:                        12:00:30   Log-Likelihood:                 127.78
No. Observations:                 420   AIC:                            -251.6
Df Residuals:                     418   BIC:                            -243.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3495      0.026     13.599      0.000       0.299       0.400
white         -0.2590      0.066     -3.895      0.000      -0.390      -0.128
==============================================================================
Omnibus:                       27.000   Durbin-Watson:                   1.849
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.814
Skew:                           0.570   Prob(JB):                     1.50e-06
Kurtosis:                       2.520   Cond. No.                         8.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [100]:
df.loc[:,'college_deglvl'] = df.college + " - " + df.deglvl

In [101]:
top_college_deglvl = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_deglvl).most_common() if t[1] >= 100 and t[0].endswith(" - 000") == False]
top_college_deglvl

[('Northern_Virginia - A', 0.25838763444991797),
 ('Tidewater - A', 0.17285111998537248),
 ('J._Sargeant_Reynolds - A', 0.06556412654920489),
 ('Thomas_Nelson - A', 0.061838350058695556),
 ('John_Tyler - A', 0.04007992135060723),
 ('Virginia_Western - A', 0.03249322284966254),
 ('Germanna - A', 0.030868826698394948),
 ('Lord_Fairfax - A', 0.02456998550498051),
 ('Central_Virginia - A', 0.02255340235471194),
 ('Piedmont_Virginia - A', 0.022505703935914525),
 ('Blue_Ridge - A', 0.021305293729512866),
 ('New_River - A', 0.019127065937764164),
 ('Southside_Virginia - A', 0.012891822636079613),
 ('Wytheville - A', 0.012383039502240501),
 ('Tidewater - C', 0.012242594158003663),
 ('Southwest_Virginia - A', 0.011227677802480848),
 ('Rappahannock - A', 0.011145530525663075),
 ('Mountain_Empire - A', 0.010562549851472423),
 ('Patrick_Henry - A', 0.01053340081776289),
 ('Virginia_Highlands - A', 0.00926409289532052),
 ('Danville - A', 0.008975252470380607),
 ('Southside_Virginia - C', 0.00892490

In [102]:
top_college_deglvl = pd.DataFrame(top_college_deglvl, columns=['college_deglvl', 'N'])
by_college_deglvl_1 = df.merge(top_college_deglvl, how='inner', on=['college_deglvl'])
by_college_deglvl_1 = by_college_deglvl_1.groupby(['college_deglvl', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='college_deglvl', columns='race', values='graduate').reset_index().round(4)
by_college_deglvl_1 = by_college_deglvl_1.merge(top_college_deglvl, how='inner', on=['college_deglvl']).sort_values(['N'], ascending=False).loc[:,['college_deglvl','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_deglvl_2 = df.merge(top_college_deglvl, how='inner', on=['college_deglvl'])
by_college_deglvl_2 = by_college_deglvl_2.groupby(['college_deglvl', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_deglvl', columns='race', values='vccsid').reset_index().round(4)
by_college_deglvl_2.loc[:,'total'] = by_college_deglvl_2.iloc[:,1:].sum(axis=1)
for v in by_college_deglvl_2.columns.values[1:-1]:
    by_college_deglvl_2.loc[:,v] = by_college_deglvl_2[v] / by_college_deglvl_2.total
by_college_deglvl_2 = by_college_deglvl_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_deglvl_2 = by_college_deglvl_2.loc[:,['college_deglvl','N','afam','asian','hisp','other','white']].fillna(0)

In [103]:
X = by_college_deglvl_1.white
X = add_constant(X, prepend=True)
y = by_college_deglvl_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.336
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.252
Time:                        12:00:38   Log-Likelihood:                 26.525
No. Observations:                  73   AIC:                            -49.05
Df Residuals:                      71   BIC:                            -44.47
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3124      0.086      3.641      0.001       0.141       0.484
white         -0.2506      0.217     -1.156      0.252      -0.683       0.182
==============================================================================
Omnibus:                        8.001   Durbin-Watson:                   2.245
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                5.716
Skew:                           0.551   Prob(JB):                       0.0574
Kurtosis:                       2.186   Cond. No.                         12.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [104]:
df.loc[:,'deglvl_cip'] = df.deglvl + " - " + df.cip.astype(str)
df.loc[:,'deglvl_curr'] = df.deglvl + " - " + df.curr

In [105]:
top_deglvl_cip = [(t[0], t[1]/df.shape[0]) for t in Counter(df.deglvl_cip).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
top_deglvl_cip

[('A - 24', 0.3067114325160121),
 ('A - 52', 0.14359078996531266),
 ('A - 51', 0.10085035681067173),
 ('A - 45', 0.06598546258191541),
 ('A - 30', 0.056827366172811374),
 ('C - 51', 0.05370841956589139),
 ('A - 11', 0.041987858102503904),
 ('A - 43', 0.02819241542142878),
 ('A - 15', 0.02500987092277891),
 ('A - 50', 0.02352327020359275),
 ('A - 14', 0.023427873365997918),
 ('A - 19', 0.015923322141870998),
 ('A - 47', 0.014661963955894862),
 ('C - 47', 0.007467452453951151),
 ('A - 12', 0.0074118042986874985),
 ('C - 48', 0.006767875644922371),
 ('A - 22', 0.00583510656621733),
 ('C - 52', 0.004653245744903557),
 ('C - 11', 0.0030606485395009157),
 ('C - 49', 0.0028831044250883084),
 ('C - 24', 0.002875154688622072),
 ('C - 19', 0.002867204952155836),
 ('A - 46', 0.00265256206756746),
 ('C - 15', 0.002591614087992983),
 ('A - 1', 0.0021649782309716434),
 ('D - 47', 0.0021543785823499952),
 ('A - 16', 0.0020801810419984576),
 ('A - 4', 0.0018284393872343133),
 ('C - 46', 0.001786040792

In [106]:
top_deglvl_cip = pd.DataFrame(top_deglvl_cip, columns=['deglvl_cip', 'N'])
by_deglvl_cip_1 = df.merge(top_deglvl_cip, how='inner', on=['deglvl_cip'])
by_deglvl_cip_1 = by_deglvl_cip_1.groupby(['deglvl_cip', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='deglvl_cip', columns='race', values='graduate').reset_index().round(4)
by_deglvl_cip_1 = by_deglvl_cip_1.merge(top_deglvl_cip, how='inner', on=['deglvl_cip']).sort_values(['N'], ascending=False).loc[:,['deglvl_cip','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_deglvl_cip_2 = df.merge(top_deglvl_cip, how='inner', on=['deglvl_cip'])
by_deglvl_cip_2 = by_deglvl_cip_2.groupby(['deglvl_cip', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='deglvl_cip', columns='race', values='vccsid').reset_index().round(4)
by_deglvl_cip_2.loc[:,'total'] = by_deglvl_cip_2.iloc[:,1:].sum(axis=1)
for v in by_deglvl_cip_2.columns.values[1:-1]:
    by_deglvl_cip_2.loc[:,v] = by_deglvl_cip_2[v] / by_deglvl_cip_2.total
by_deglvl_cip_2 = by_deglvl_cip_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_deglvl_cip_2 = by_deglvl_cip_2.loc[:,['deglvl_cip','N','afam','asian','hisp','other','white']].fillna(0)

In [107]:
X = by_deglvl_cip_1.white
X = add_constant(X, prepend=True)
y = by_deglvl_cip_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     4.936
Date:                Mon, 16 May 2022   Prob (F-statistic):             0.0323
Time:                        12:03:17   Log-Likelihood:                 37.398
No. Observations:                  40   AIC:                            -70.80
Df Residuals:                      38   BIC:                            -67.42
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4029      0.068      5.929      0.000       0.265       0.540
white         -0.4398      0.198     -2.222      0.032      -0.841      -0.039
==============================================================================
Omnibus:                        0.247   Durbin-Watson:                   1.599
Prob(Omnibus):                  0.884   Jarque-Bera (JB):                0.168
Skew:                          -0.145   Prob(JB):                        0.919
Kurtosis:                       2.872   Cond. No.                         14.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [108]:
top_deglvl_curr = [(t[0], t[1]/df.shape[0]) for t in Counter(df.deglvl_curr).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
top_deglvl_curr

[('A - 699', 0.11178124445174642),
 ('A - 213', 0.07650296392674583),
 ('A - 697', 0.0662902024797878),
 ('A - 882', 0.06598546258191541),
 ('A - 880', 0.056827366172811374),
 ('A - 156', 0.05534871519009145),
 ('A - 648', 0.049357263806704806),
 ('A - 212', 0.0361368520633541),
 ('A - 649', 0.027670382726812607),
 ('N - 000', 0.02667666566853309),
 ('C - 190', 0.02573594685336181),
 ('A - 831', 0.023427873365997918),
 ('A - 299', 0.02207111834242695),
 ('C - 000', 0.02110125049354614),
 ('A - 400', 0.019972387915340607),
 ('A - 636', 0.015923322141870998),
 ('A - 881', 0.014934904907902303),
 ('A - 000', 0.014675213516671922),
 ('A - 203', 0.014415522125441541),
 ('A - 216', 0.01425917730827223),
 ('A - 480', 0.013138264466532934),
 ('A - 340', 0.011696712253988781),
 ('A - 624', 0.01133367428869733),
 ('C - 157', 0.011251527011879555),
 ('A - 650', 0.009727827522517628),
 ('A - 909', 0.009261442983165108),
 ('A - 298', 0.009105098165995798),
 ('A - 146', 0.0071574127317679415),
 ('A 

In [109]:
top_deglvl_curr = pd.DataFrame(top_deglvl_curr, columns=['deglvl_curr', 'N'])
by_deglvl_curr_1 = df.merge(top_deglvl_curr, how='inner', on=['deglvl_curr'])
by_deglvl_curr_1 = by_deglvl_curr_1.groupby(['deglvl_curr', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='deglvl_curr', columns='race', values='graduate').reset_index().round(4)
by_deglvl_curr_1 = by_deglvl_curr_1.merge(top_deglvl_curr, how='inner', on=['deglvl_curr']).sort_values(['N'], ascending=False).loc[:,['deglvl_curr','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_deglvl_curr_2 = df.merge(top_deglvl_curr, how='inner', on=['deglvl_curr'])
by_deglvl_curr_2 = by_deglvl_curr_2.groupby(['deglvl_curr', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='deglvl_curr', columns='race', values='vccsid').reset_index().round(4)
by_deglvl_curr_2.loc[:,'total'] = by_deglvl_curr_2.iloc[:,1:].sum(axis=1)
for v in by_deglvl_curr_2.columns.values[1:-1]:
    by_deglvl_curr_2.loc[:,v] = by_deglvl_curr_2[v] / by_deglvl_curr_2.total
by_deglvl_curr_2 = by_deglvl_curr_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_deglvl_curr_2 = by_deglvl_curr_2.loc[:,['deglvl_curr','N','afam','asian','hisp','other','white']].fillna(0)

In [110]:
X = by_deglvl_curr_1.white
X = add_constant(X, prepend=True)
y = by_deglvl_curr_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.857
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.177
Time:                        12:04:50   Log-Likelihood:                 69.512
No. Observations:                  86   AIC:                            -135.0
Df Residuals:                      84   BIC:                            -130.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3225      0.046      6.955      0.000       0.230       0.415
white         -0.1759      0.129     -1.363      0.177      -0.433       0.081
==============================================================================
Omnibus:                        3.647   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.161   Jarque-Bera (JB):                3.508
Skew:                           0.490   Prob(JB):                        0.173
Kurtosis:                       2.869   Cond. No.                         12.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
print("\n\n\nBy college/course/cip")
df.loc[:,'college_course_cip'] = df.college + " - " + df.course + " - " + df.cip.astype(str)
df.loc[:,'college_course_curr'] = df.college + " - " + df.course + " - " + df.curr
top_college_course_cip = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_course_cip).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
print(top_college_course_cip)
top_college_course_cip = pd.DataFrame(top_college_course_cip, columns=['college_course_cip', 'N'])
by_college_course_cip_1 = df.merge(top_college_course_cip, how='inner', on=['college_course_cip'])
by_college_course_cip_1 = by_college_course_cip_1.groupby(['college_course_cip', 'race']).agg({'grade':'mean'}).reset_index().pivot(index='college_course_cip', columns='race', values='grade').reset_index().round(4)
by_college_course_cip_1 = by_college_course_cip_1.merge(top_college_course_cip, how='inner', on=['college_course_cip']).sort_values(['N'], ascending=False).loc[:,['college_course_cip','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_course_cip_2 = df.merge(top_college_course_cip, how='inner', on=['college_course_cip'])
by_college_course_cip_2 = by_college_course_cip_2.groupby(['college_course_cip', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_course_cip', columns='race', values='vccsid').reset_index().round(4)
by_college_course_cip_2.loc[:,'total'] = by_college_course_cip_2.iloc[:,1:].sum(axis=1)
for v in by_college_course_cip_2.columns.values[1:-1]:
    by_college_course_cip_2.loc[:,v] = by_college_course_cip_2[v] / by_college_course_cip_2.total
by_college_course_cip_2 = by_college_course_cip_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_course_cip_2 = by_college_course_cip_2.loc[:,['college_course_cip','N','afam','asian','hisp','other','white']].fillna(0)
X = by_college_course_cip_1.white
X = add_constant(X, prepend=True)
y = by_college_course_cip_2.afam
model = sm.OLS(y,X)
results = model.fit()
print(results.summary())


print("\n\n\nBy college/course/curr")
top_college_course_curr = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_course_curr).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
print(top_college_course_curr)
top_college_course_curr = pd.DataFrame(top_college_course_curr, columns=['college_course_curr', 'N'])
by_college_course_curr_1 = df.merge(top_college_course_curr, how='inner', on=['college_course_curr'])
by_college_course_curr_1 = by_college_course_curr_1.groupby(['college_course_curr', 'race']).agg({'grade':'mean'}).reset_index().pivot(index='college_course_curr', columns='race', values='grade').reset_index().round(4)
by_college_course_curr_1 = by_college_course_curr_1.merge(top_college_course_curr, how='inner', on=['college_course_curr']).sort_values(['N'], ascending=False).loc[:,['college_course_curr','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_course_curr_2 = df.merge(top_college_course_curr, how='inner', on=['college_course_curr'])
by_college_course_curr_2 = by_college_course_curr_2.groupby(['college_course_curr', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_course_curr', columns='race', values='vccsid').reset_index().round(4)
by_college_course_curr_2.loc[:,'total'] = by_college_course_curr_2.iloc[:,1:].sum(axis=1)
for v in by_college_course_curr_2.columns.values[1:-1]:
    by_college_course_curr_2.loc[:,v] = by_college_course_curr_2[v] / by_college_course_curr_2.total
by_college_course_curr_2 = by_college_course_curr_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_course_curr_2 = by_college_course_curr_2.loc[:,['college_course_curr','N','afam','asian','hisp','other','white']].fillna(0)
X = by_college_course_curr_1.white
X = add_constant(X, prepend=True)
y = by_college_course_curr_2.afam
model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

In [111]:
df.loc[:,'college_deglvl_cip'] = df.college + " - " + df.deglvl + " - " + df.cip.astype(str)
df.loc[:,'college_deglvl_curr'] = df.college + " - " + df.deglvl + " - " + df.curr

In [112]:
top_college_deglvl_cip = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_deglvl_cip).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
top_college_deglvl_cip

[('Northern_Virginia - A - 24', 0.09497815147427863),
 ('Northern_Virginia - A - 52', 0.055282467386206145),
 ('Tidewater - A - 51', 0.031160317035490274),
 ('Tidewater - A - 52', 0.03012420138272416),
 ('Tidewater - A - 45', 0.02587639219759865),
 ('Tidewater - A - 24', 0.024418940512122022),
 ('Germanna - A - 24', 0.023199980920632482),
 ('Northern_Virginia - A - 11', 0.022468605165738755),
 ('John_Tyler - A - 24', 0.020303626934767113),
 ('Lord_Fairfax - A - 24', 0.01924896189691312),
 ('Tidewater - A - 30', 0.018766677884628123),
 ('Northern_Virginia - A - 45', 0.018671281047033292),
 ('Northern_Virginia - A - 30', 0.01705218472007653),
 ('J._Sargeant_Reynolds - A - 51', 0.016622898950899776),
 ('Central_Virginia - A - 24', 0.01553908487933625),
 ('Blue_Ridge - A - 24', 0.01399948591704185),
 ('Thomas_Nelson - A - 52', 0.013787492944608886),
 ('J._Sargeant_Reynolds - A - 52', 0.012372439853618853),
 ('New_River - A - 24', 0.01182390803744856),
 ('Northern_Virginia - A - 50', 0.0111

In [113]:
top_college_deglvl_cip = pd.DataFrame(top_college_deglvl_cip, columns=['college_deglvl_cip', 'N'])
by_college_deglvl_cip_1 = df.merge(top_college_deglvl_cip, how='inner', on=['college_deglvl_cip'])
by_college_deglvl_cip_1 = by_college_deglvl_cip_1.groupby(['college_deglvl_cip', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='college_deglvl_cip', columns='race', values='graduate').reset_index().round(4)
by_college_deglvl_cip_1 = by_college_deglvl_cip_1.merge(top_college_deglvl_cip, how='inner', on=['college_deglvl_cip']).sort_values(['N'], ascending=False).loc[:,['college_deglvl_cip','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_deglvl_cip_2 = df.merge(top_college_deglvl_cip, how='inner', on=['college_deglvl_cip'])
by_college_deglvl_cip_2 = by_college_deglvl_cip_2.groupby(['college_deglvl_cip', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_deglvl_cip', columns='race', values='vccsid').reset_index().round(4)
by_college_deglvl_cip_2.loc[:,'total'] = by_college_deglvl_cip_2.iloc[:,1:].sum(axis=1)
for v in by_college_deglvl_cip_2.columns.values[1:-1]:
    by_college_deglvl_cip_2.loc[:,v] = by_college_deglvl_cip_2[v] / by_college_deglvl_cip_2.total
by_college_deglvl_cip_2 = by_college_deglvl_cip_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_deglvl_cip_2 = by_college_deglvl_cip_2.loc[:,['college_deglvl_cip','N','afam','asian','hisp','other','white']].fillna(0)

In [114]:
X = by_college_deglvl_cip_1.white
X = add_constant(X, prepend=True)
y = by_college_deglvl_cip_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.3762
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.540
Time:                        12:08:38   Log-Likelihood:                 94.368
No. Observations:                 269   AIC:                            -184.7
Df Residuals:                     267   BIC:                            -177.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2721      0.034      8.088      0.000       0.206       0.338
white         -0.0561      0.091     -0.613      0.540      -0.236       0.124
==============================================================================
Omnibus:                       18.517   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               14.904
Skew:                           0.484   Prob(JB):                     0.000580
Kurtosis:                       2.375   Cond. No.                         9.86
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
top_college_deglvl_curr = [(t[0], t[1]/df.shape[0]) for t in Counter(df.college_deglvl_curr).most_common() if t[1] >= 100 and t[0].endswith(" - 0") == False]
top_college_deglvl_curr

[('Northern_Virginia - A - 699', 0.067699955746467),
 ('Northern_Virginia - A - 213', 0.029713464998635294),
 ('Northern_Virginia - A - 648', 0.02727819572781162),
 ('Tidewater - A - 882', 0.02587639219759865),
 ('Tidewater - A - 213', 0.02046792148840266),
 ('Tidewater - A - 880', 0.018766677884628123),
 ('Northern_Virginia - A - 882', 0.018671281047033292),
 ('Northern_Virginia - A - 880', 0.01705218472007653),
 ('Tidewater - A - 699', 0.017012436037745347),
 ('Tidewater - A - 156', 0.015395989622943999),
 ('Northern_Virginia - A - 212', 0.015297942873193754),
 ('Blue_Ridge - A - 649', 0.013983586444109377),
 ('Germanna - A - 697', 0.013941187849622784),
 ('John_Tyler - A - 699', 0.012125998023165533),
 ('Central_Virginia - A - 697', 0.010949437026162582),
 ('Northern_Virginia - A - 340', 0.010822241242702804),
 ('Thomas_Nelson - A - 882', 0.010514851432675007),
 ('J._Sargeant_Reynolds - A - 213', 0.009603281651213262),
 ('Thomas_Nelson - A - 213', 0.0090838988687525),
 ('J._Sargeant

In [116]:
top_college_deglvl_curr = pd.DataFrame(top_college_deglvl_curr, columns=['college_deglvl_curr', 'N'])
by_college_deglvl_curr_1 = df.merge(top_college_deglvl_curr, how='inner', on=['college_deglvl_curr'])
by_college_deglvl_curr_1 = by_college_deglvl_curr_1.groupby(['college_deglvl_curr', 'race']).agg({'graduate':'mean'}).reset_index().pivot(index='college_deglvl_curr', columns='race', values='graduate').reset_index().round(4)
by_college_deglvl_curr_1 = by_college_deglvl_curr_1.merge(top_college_deglvl_curr, how='inner', on=['college_deglvl_curr']).sort_values(['N'], ascending=False).loc[:,['college_deglvl_curr','N','afam','asian','hisp','other','white']].drop(['N'], axis=1)
by_college_deglvl_curr_2 = df.merge(top_college_deglvl_curr, how='inner', on=['college_deglvl_curr'])
by_college_deglvl_curr_2 = by_college_deglvl_curr_2.groupby(['college_deglvl_curr', 'race']).agg({'vccsid':'count'}).reset_index().pivot(index='college_deglvl_curr', columns='race', values='vccsid').reset_index().round(4)
by_college_deglvl_curr_2.loc[:,'total'] = by_college_deglvl_curr_2.iloc[:,1:].sum(axis=1)
for v in by_college_deglvl_curr_2.columns.values[1:-1]:
    by_college_deglvl_curr_2.loc[:,v] = by_college_deglvl_curr_2[v] / by_college_deglvl_curr_2.total
by_college_deglvl_curr_2 = by_college_deglvl_curr_2.rename(columns = {'total': 'N'}).round(4).sort_values(['N'], ascending=False)
by_college_deglvl_curr_2 = by_college_deglvl_curr_2.loc[:,['college_deglvl_curr','N','afam','asian','hisp','other','white']].fillna(0)

In [117]:
X = by_college_deglvl_curr_1.white
X = add_constant(X, prepend=True)
y = by_college_deglvl_curr_2.afam

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afam   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     18.77
Date:                Mon, 16 May 2022   Prob (F-statistic):           1.80e-05
Time:                        12:10:17   Log-Likelihood:                 148.79
No. Observations:                 475   AIC:                            -293.6
Df Residuals:                     473   BIC:                            -285.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3487      0.024     14.241      0.000       0.301       0.397
white         -0.2730      0.063     -4.332      0.000      -0.397      -0.149
==============================================================================
Omnibus:                       30.966   Durbin-Watson:                   1.722
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.846
Skew:                           0.593   Prob(JB):                     1.22e-07
Kurtosis:                       2.549   Cond. No.                         8.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""